In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from scipy.integrate import odeint
import Global as gl

In [2]:
# Set a seed for reproducibility
np.random.seed(gl.seed_value)
model_name = gl.model_name

# Reality ODE
We will have 4D reality to test :

$\frac{dX^\epsilon}{dt} = \begin{pmatrix} \frac{dV^\epsilon}{dt} \\ \\ \frac{dW^\epsilon}{dt} \end{pmatrix} $


$X^{\epsilon}(0) = x = [x_1,x_2,x_3,x_4]^T$

$\frac{dX^\epsilon}{dt} =A_1X^\epsilon +tanh(||A_3X^\epsilon||_2)A_2 + \frac{1}{\epsilon} B X^\epsilon$

$ A_1=\begin{pmatrix}
    0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
\end{pmatrix} \quad A_2=\begin{pmatrix}
    1 \\ 0 \\ 0 \\0 
\end{pmatrix} A_3=\begin{pmatrix}
    0 & 0 & 1 & 0\\
    0 & 0 & 0 & 1\\
    0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\
\end{pmatrix} \quad B=\begin{pmatrix}
    0 & -1 & 0 & 0\\
    1 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\  
\end{pmatrix}$

# System 
In terms of discrete dynamical system formulation we have :

$\begin{pmatrix}
    V_1^{\epsilon}\\ \\ V_2^{\epsilon}\\ \\ X_1^{\epsilon}\\ \\X_2^{\epsilon}
\end{pmatrix}  = 
\begin{pmatrix}
 tanh(\sqrt{(X_1^{\epsilon})^2+(X_2^{\epsilon})^2})-\frac{1}{\epsilon}V_2^{\epsilon}  \\ \\  \frac{1}{\epsilon}V_1^{\epsilon}\\ \\  V_1^{\epsilon}\\ \\ V_2^{\epsilon}
\end{pmatrix} $

# ODE Analytical Solution
The solution to the previosuly mentioned problem is :

$X^{\epsilon}(t,\tau) =   \begin{pmatrix}
    x_1 \cos(\tau) - x_2 sin(\tau) +\epsilon tanh(\sqrt{x_3^2+x_4^2}) sin(\tau)+?\\ \\
    x_1 sin(\tau) + x_2 \cos(\tau) +\epsilon tanh(\sqrt{x_3^2+x_4^2})(1-cos(\tau))+?\\ \\
    \epsilon x_1 sin(\tau)+\epsilon x_2(cos(\tau)-1)+ x_3 + \epsilon^2tanh(\sqrt{x_3^2+x_4^2})(1-cos(\tau))\\ \\
    \epsilon x1(1-cos(\tau)) +\epsilon x_2sin(\tau) +x_4-\epsilon^2 tanh(\sqrt{x_3^2+x_4^2})sin(\tau) +\epsilon   tanh(\sqrt{x_3^2+x_4^2}) t
\end{pmatrix} $

# ODE Two scale expansion
The expansion to the previosuly mentioned problem is :

$X^\epsilon(t,\tau)= X^0(t,\tau)+\epsilon X^1(t,\tau)+\epsilon^2 X^2(t,\tau)...$

$\begin{pmatrix}
    V^\epsilon_1\\ \\
    V^\epsilon_2\\ \\ 
    W^\epsilon_1\\ \\
    W^\epsilon_2\\ \\
\end{pmatrix} = \begin{pmatrix}
    x_1 \cos(\tau) - x_2 sin(\tau)\\ \\
    x_1 sin(\tau) + x_2 \cos(\tau)\\ \\
    x_3\\ \\
    x_4\\ \\
\end{pmatrix} + \epsilon \begin{pmatrix}
    tanh(\sqrt{x_3^2+x_4^2})sin(\tau)\\ \\
    tanh(\sqrt{x_3^2+x_4^2})(1-cos(\tau))\\ \\
    x_1 sin(\tau)+x_2(cos(\tau)-1)\\ \\
    x1(1-cos(\tau)) +x_2sin(\tau) 
\end{pmatrix}+ \\ \\ \epsilon^2 \begin{pmatrix}
        ? \\ \\ ? \\ \\ tanh(\sqrt{x_3^2+x_4^2})(1-cos(\tau)) \\ \\ tanh(\sqrt{x_3^2+x_4^2})(\tau -sin(\tau))
    \end{pmatrix}$

## True macroscopic system 

$Y^0(t)=\begin{pmatrix}
x_1 \\ \\
x_2  \\ \\
x_3 \\ \\
x_4
\end{pmatrix} \quad X^0(t,\tau) = \begin{pmatrix}
cos(\tau) & -sin(\tau) & 0 & 0\\ \\
sin(\tau) & cos(\tau)  & 0 & 0\\ \\
0 & 0  & 1 & 0\\ \\
0 & 0  & 0 & 1\\ \\
\end{pmatrix}\begin{pmatrix}
x_1 \\ \\
x_2  \\ \\
x_3 \\ \\
x_4
\end{pmatrix}$






# Parameters
$\begin{cases}
\epsilon = 1/50\\
w = 60s\\
T = 15 hours\\
\Delta t=30minutes\\
\Delta t_\epsilon=10^{-2}seconds\\
h = \frac{1s}{\Delta t_\epsilon}\\
\mu_0,\sigma_0=0,0.01\\
\end{cases}$

## Reality 


In [3]:
def f(x3,x4):
    return float(np.tanh(np.sqrt(x3**2+x4**2)))

In [4]:
def Ue(t,eps,x1,x2,x3,x4):
    tau = t / eps
    result = [
        x1 * np.cos(tau) - x2 * np.sin(tau) + eps * f(x3,x4)*np.sin(tau),
        x1 * np.sin(tau) + x2 * np.cos(tau) + eps *f(x3,x4)* (1 - np.cos(tau)),
        eps * x1 * np.sin(tau) + eps * x2 * (np.cos(tau) - 1) + x3 + (eps**2) *f(x3,x4)* (1 - np.cos(tau)),
        eps * x1 * (1 - np.cos(tau)) + eps * x2 * np.sin(tau) + x4 - (eps**2) *f(x3,x4)* np.sin(tau) + eps *f(x3,x4)* t
    ]
    return result

def U0(x1,x2,x3,x4):
    return [x1,x2,x3,x4]


# Data Generation

I begin by creating a dataframe containing the generated values across a period of "h" hours where the data is generated each "step" seconds meaning $\Delta t=step$ .

* Ue the microscopic analytical solution.
* U0 the macroscopic analytical solution.
* eps is the microstructure ratio $\epsilon$.
* h number of hours.
* step is the time step proportional to $\epsilon$.
* window is the rolling average time window in case we don't want to use U0.


In [5]:
def genData(Ue,U0,eps,h,step,x1,x2,x3,x4):
    #generate the microscopic data using Ue h hours each step time
    v=[]
    for i in range(1,h*3600+101):
        for j in range(int(1/step)):
            micro_state = Ue(i+step*j,eps,x1,x2,x3,x4)
            v.append([i+step*j,micro_state[0],micro_state[1],micro_state[2],micro_state[3]])

    #save results in pandas
    arr= np.array(v)
    df = pd.DataFrame(arr)
    df.columns = ['t','Ve1','Ve2','Xe1','Xe2']

    #generate macroscopic data using U0
    macro_state = U0(x1,x2,x3,x4)
    df['V01'] = macro_state[0]
    df['V02'] = macro_state[1]
    df['X01'] = macro_state[2]
    df['X02'] = macro_state[3]
    return df


## Rolling average generator

In [6]:
#Data generator using moving average with time window instead of U0 
def genDatawithMean(Ue,x1,x2,x3,x4,window,eps,h,step):
    v=[]
    for i in range(1,h*3600+101):
        for j in range(int(1/step)):
            state = Ue(i+step*j,eps,x1,x2,x3,x4)
            v.append([i+step*j,state[0],state[1],state[2],state[3]])

    arr= np.array(v)
    df = pd.DataFrame(arr)
    df.columns = ['t','Ve1','Ve2','Xe1','Xe2']

    #generate macroscopic values by taking a rolling mean of window time
    df['V01'] = df['Ve1'].rolling(window=window*(int(1/step))).mean()
    df['V02'] = df['Ve2'].rolling(window=window*(int(1/step))).mean()
    df['X01'] = df['Xe1'].rolling(window=window*(int(1/step))).mean()
    df['X02'] = df['Xe2'].rolling(window=window*(int(1/step))).mean()
    return df

## ODE Solver Generator

In [7]:
# Define the system of ODEs
def ode_system(y, t, eps):
    V1, V2, X1, X2 = y
    dV1_dt = np.tanh(np.sqrt(X1**2 + X2**2)) - V2 / eps
    dV2_dt = V1 / eps
    dX1_dt = V1
    dX2_dt = V2
    return [dV1_dt, dV2_dt, dX1_dt, dX2_dt]

def genDatawithOdeSolver(Ue,x1,x2,x3,x4,window,eps,h,step):
    v=[]
    t = np.arange(1, h * 3600+101, step)
    initial_conditions = [x1,x2,x3,x4]
    # Solve the ODEs
    solution = odeint(ode_system, initial_conditions, t, args=(eps,))
    df = pd.DataFrame(solution, columns=['Ve1', 'Ve2', 'Xe1', 'Xe2'])
    df['t'] = t

    #generate macroscopic values by taking a rolling mean of window time
    df['V01'] = df['Ve1'].rolling(window=window*(int(1/step))).mean()
    df['V02'] = df['Ve2'].rolling(window=window*(int(1/step))).mean()
    df['X01'] = df['Xe1'].rolling(window=window*(int(1/step))).mean()
    df['X02'] = df['Xe2'].rolling(window=window*(int(1/step))).mean()
    return df

## Data Noising
This function adds Gaussian white noise to the macroscopic data $N(\mu,\sigma)$

In [8]:
def genNoiseData(df,gaussian_noise):
    # Add Gaussian noise to the macroscopic data columns
    df['V01']=df['V01'].add(gaussian_noise) 
    df['V02']=df['V02'].add(gaussian_noise) 
    df['X01']=df['X01'].add(gaussian_noise) 
    df['X02']=df['X02'].add(gaussian_noise) 

## Data Points
At each macrostep take a single macroscopic data   and all the data coming from the next microscopic window 

In [9]:
def interval(df,macrostep,microstep):
    
    vals=[]
    # Define the macroscopic interval in seconds (macrostep*60 = 30 minutes)
    interval_seconds = macrostep * 60

    # Initialize the starting time
    current_time = 0

    while current_time < df['t'].max():

        # Find the index where 'X' is greater than or equal to the current_time
        index = df[df['t'] >= current_time].index[0]
        
        # Extract the 'macro' value at the current index
        current_t = df.at[index, 't']
        print("index ",current_t)
        current_v01 = float(df.at[index, 'V01'])
        current_v02 = float(df.at[index, 'V02'])
        current_x01 = float(df.at[index, 'X01'])
        current_x02 = float(df.at[index, 'X02'])
        

        # Extract the next microstep values of 'the micro' from the current index
        next_microsteps_ve1 = df.loc[index:index + (microstep-1), 'Ve1'].tolist()
        next_microsteps_ve2 = df.loc[index:index + (microstep-1), 'Ve2'].tolist()
        next_microsteps_xe1 = df.loc[index:index + (microstep-1), 'Xe1'].tolist()
        next_microsteps_xe2 = df.loc[index:index + (microstep-1), 'Xe2'].tolist()

        vals.append([current_t,current_v01,next_microsteps_ve1,current_v02,next_microsteps_ve2,current_x01,next_microsteps_xe1,current_x02,next_microsteps_xe2])


        # Update the current time to the next interval
        current_time += interval_seconds
        
    vals = vals[1:]
    return vals

# JSON

In [10]:
def tojson(vals,name):
    file_path = name
    # Write the data to a JSON file
    with open(file_path, "w") as json_file:
        json.dump(vals, json_file)

    print(f"Data has been saved to {file_path}")


# Initialization

In [11]:
# Micro structure ratio
eps=gl.eps

# Number of Hours for generation
h=gl.h

#time step in seconds
step=gl.step

# Macroscopic noise of mu mean and  sigma standard deviation 
mu,sigma=gl.mu,gl.sigma

#Macroscopic step  in minutes
macrostep=gl.macrostep

# Number of captured  microscopic steps in 10 seconds per macroscpic value
microstep=gl.microstep/gl.step

# Generation 
## Initial Params

In [12]:
#rolling average window in seconds =1min
w=gl.w
# initial state x1,x2,x3,x4
x_0=gl.x_0

## Full True Data

In [13]:
#df1=genData(Ue,U0,eps,h,step,x_0[0],x_0[1],x_0[2],x_0[3])
#df1=genDatawithMean(Ue,x_0[0],x_0[1],x_0[2],x_0[3],w,eps,h,step)
df1=genDatawithOdeSolver(Ue,x_0[0],x_0[1],x_0[2],x_0[3],w,eps,h,step)
df1.to_csv(("GenData/True/Full/"+model_name+".csv"), index=False)

## True Cut Data

In [14]:
#Cut True Data
#Taking measurements at intervals
vals1=interval(df1,macrostep,microstep)


index  1.0
index  1800.0000000000016
index  3600.000000000003
index  5400.000000000005
index  7200.000000000006
index  9000.000000000007
index  10800.00000000001
index  12600.000000000011
index  14400.000000000013
index  16200.000000000015
index  18000.000000000015
index  19800.00000000002
index  21600.00000000002
index  23400.000000000022
index  25200.000000000022
index  27000.000000000025
index  28800.000000000025
index  30600.000000000025
index  32400.00000000003
index  34200.00000000003
index  36000.00000000003
index  37800.00000000004
index  39600.00000000004
index  41400.00000000004
index  43200.00000000004
index  45000.00000000004
index  46800.000000000044
index  48600.000000000044
index  50400.000000000044
index  52200.000000000044
index  54000.00000000005


In [15]:
print(type(vals1[0][2]))
# Saving clean but cut data in JSON
tojson(vals1,("GenData/True/Cut/"+model_name+".json"))

<class 'list'>
Data has been saved to GenData/True/Cut/4D_Tokamak.json


## Adding noise to data

In [16]:
gaussian_noise = np.random.normal(mu, sigma, len(df1))
df1_noised=df1.copy()
genNoiseData(df1_noised,gaussian_noise)

## Noised Full Data

In [17]:
df1_noised.to_csv(("GenData/Noised/Full/"+model_name+".csv"), index=False)

## Noised Cut Data

In [18]:
#Taking measurements
vals1=interval(df1_noised,macrostep,microstep)
# Saving noised cut data in JSON
tojson(vals1,("GenData/Noised/Cut/"+model_name+".json"))


index  1.0
index  1800.0000000000016
index  3600.000000000003
index  5400.000000000005
index  7200.000000000006
index  9000.000000000007
index  10800.00000000001
index  12600.000000000011
index  14400.000000000013
index  16200.000000000015
index  18000.000000000015
index  19800.00000000002
index  21600.00000000002
index  23400.000000000022
index  25200.000000000022
index  27000.000000000025
index  28800.000000000025
index  30600.000000000025
index  32400.00000000003
index  34200.00000000003
index  36000.00000000003
index  37800.00000000004
index  39600.00000000004
index  41400.00000000004
index  43200.00000000004
index  45000.00000000004
index  46800.000000000044
index  48600.000000000044
index  50400.000000000044
index  52200.000000000044
index  54000.00000000005
Data has been saved to GenData/Noised/Cut/4D_Tokamak.json
